In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings


In [2]:
labels=['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite','Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']

In [22]:
X_train=[]
y_train=[]


for i in labels:
    
    folderpath=os.path.join('dataset/archive/PlantVillage/',i)
    print(folderpath)
    for j in os.listdir(folderpath):
        img=cv2.imread(os.path.join(folderpath,j))
        img=cv2.resize(img,(244,244))
        X_train.append(img)
        y_train.append(i)

dataset/archive/PlantVillage/Pepper__bell___Bacterial_spot
dataset/archive/PlantVillage/Pepper__bell___healthy
dataset/archive/PlantVillage/Potato___Early_blight
dataset/archive/PlantVillage/Potato___Late_blight
dataset/archive/PlantVillage/Potato___healthy
dataset/archive/PlantVillage/Tomato_Bacterial_spot
dataset/archive/PlantVillage/Tomato_Early_blight
dataset/archive/PlantVillage/Tomato_Late_blight
dataset/archive/PlantVillage/Tomato_Leaf_Mold
dataset/archive/PlantVillage/Tomato_Septoria_leaf_spot
dataset/archive/PlantVillage/Tomato_Spider_mites_Two_spotted_spider_mite
dataset/archive/PlantVillage/Tomato__Target_Spot
dataset/archive/PlantVillage/Tomato__Tomato_mosaic_virus
dataset/archive/PlantVillage/Tomato_healthy


In [23]:
X_train=np.array(X_train)
y_train=np.array(y_train)

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.5,random_state=101)

In [25]:
y_train_new=[]


for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train=tf.keras.utils.to_categorical(y_train)



y_test_new=[]

for i in labels:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test=tf.keras.utils.to_categorical(y_test)
    
    

In [27]:
base_model = VGG16(input_shape = (244, 244, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [28]:
base_model.trainable=False

In [29]:
model1 = base_model.output

model1 = tf.keras.layers.Dense(512,activation='relu')(model1)
model1 = tf.keras.layers.GlobalAveragePooling2D()(model1)
model1 = tf.keras.layers.Dropout(rate=0.5)(model1)
model1 = tf.keras.layers.Dense(14,activation='softmax')(model1)
model1 = tf.keras.models.Model(inputs=base_model.input, outputs = model1)

In [30]:
model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [31]:
tensorboard1 = TensorBoard(log_dir = 'logs')
checkpoint1 = ModelCheckpoint("vgg16",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr1 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4, patience = 2, min_delta = 0.0001,
                              mode='auto',verbose=1)

In [32]:
model1.compile(optimizer="Adam",loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
history1 = model1.fit(X_train,y_train,validation_split=0.1, epochs =10, verbose=1, batch_size=64,
                      callbacks=[tensorboard1,checkpoint1,reduce_lr1])

Epoch 1/10
25/25 [==============================] - ETA: 0s - loss: 5.6425 - accuracy: 0.2474 
Epoch 1: val_accuracy improved from -inf to 0.67429, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


INFO:tensorflow:Assets written to: vgg16\assets


25/25 [==============================] - 535s 22s/step - loss: 5.6425 - accuracy: 0.2474 - val_loss: 0.9904 - val_accuracy: 0.6743 - lr: 0.0010
Epoch 2/10
25/25 [==============================] - ETA: 0s - loss: 1.6850 - accuracy: 0.5593 
Epoch 2: val_accuracy improved from 0.67429 to 0.80000, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


...

INFO:tensorflow:Assets written to: vgg16\assets


25/25 [==============================] - 492s 20s/step - loss: 0.9661 - accuracy: 0.6939 - val_loss: 0.5728 - val_accuracy: 0.8343 - lr: 0.0010
Epoch 4/10
25/25 [==============================] - ETA: 0s - loss: 0.7330 - accuracy: 0.7589 
Epoch 4: val_accuracy did not improve from 0.83429
25/25 [==============================] - 397s 16s/step - loss: 0.7330 - accuracy: 0.7589 - val_loss: 0.5709 - val_accuracy: 0.8229 - lr: 0.0010
Epoch 5/10
25/25 [==============================] - ETA: 0s - loss: 0.5924 - accuracy: 0.7985 
Epoch 5: val_accuracy did not improve from 0.83429

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
25/25 [==============================] - 927s 38s/step - loss: 0.5924 - accuracy: 0.7985 - val_loss: 0.4852 - val_accuracy: 0.8343 - lr: 0.0010
Epoch 6/10
25/25 [==============================] - ETA: 0s - loss: 0.4541 - accuracy: 0.8520 
Epoch 6: val_accuracy improved from 0.83429 to 0.85143, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


INFO:tensorflow:Assets written to: vgg16\assets


25/25 [==============================] - 410s 16s/step - loss: 0.4541 - accuracy: 0.8520 - val_loss: 0.4534 - val_accuracy: 0.8514 - lr: 4.0000e-04
Epoch 7/10
25/25 [==============================] - ETA: 0s - loss: 0.4649 - accuracy: 0.8469 
Epoch 7: val_accuracy did not improve from 0.85143
25/25 [==============================] - 1182s 48s/step - loss: 0.4649 - accuracy: 0.8469 - val_loss: 0.4501 - val_accuracy: 0.8400 - lr: 4.0000e-04
Epoch 8/10
25/25 [==============================] - ETA: 0s - loss: 0.4077 - accuracy: 0.8756 
Epoch 8: val_accuracy did not improve from 0.85143

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.00016000000759959222.
25/25 [==============================] - 423s 17s/step - loss: 0.4077 - accuracy: 0.8756 - val_loss: 0.4523 - val_accuracy: 0.8457 - lr: 4.0000e-04
Epoch 9/10
25/25 [==============================] - ETA: 0s - loss: 0.3944 - accuracy: 0.8699 
Epoch 9: val_accuracy did not improve from 0.85143
25/25 [==============================] 

In [34]:
model1.save('dataset/archive/')

INFO:tensorflow:Assets written to: dataset/archive/assets


INFO:tensorflow:Assets written to: dataset/archive/assets


In [35]:
model1.save_weights("model.h5")

In [47]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('dataset/archive/') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# Mobilnet

In [36]:
image_size = 244
IMG_SHAPE = (image_size, image_size, 3)

#Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')

In [37]:
base_model.trainable = False

In [38]:
from tensorflow import keras


In [39]:
model = tf.keras.Sequential([
                          base_model,
                          keras.layers.GlobalAveragePooling2D(),
                          keras.layers.Dense(14, activation='softmax')])

In [40]:
model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
 # previous one
epochs = 100
steps_per_epoch = 70
validation_steps = 70

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)


In [41]:
tensorboard1 = TensorBoard(log_dir = 'logs')
checkpoint1 = ModelCheckpoint("vgg16",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr1 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4, patience = 2, min_delta = 0.0001,
                              mode='auto',verbose=1)

In [45]:
history1 = model.fit(X_train,y_train,validation_split=0.1, epochs =1, verbose=1, batch_size=32,
                      callbacks=[tensorboard1,checkpoint1,reduce_lr1])

246/246 [==============================] - ETA: 0s - loss: 0.4135 - accuracy: 0.8818
Epoch 1: val_accuracy did not improve from 0.80275
246/246 [==============================] - 217s 880ms/step - loss: 0.4135 - accuracy: 0.8818 - val_loss: 0.6096 - val_accuracy: 0.8005 - lr: 6.5536e-07


In [32]:

img=cv2.imread('dataset/potato late blight.JPG')
img=cv2.resize(img,(244,244))
image=np.array(img)

In [47]:
u=model.predict(X_train)

273/273 [==============================] - 219s 802ms/step


In [63]:
np.argmax(u[8])

7

In [43]:
model.save('dataset/archive/')

INFO:tensorflow:Assets written to: dataset/archive/assets


INFO:tensorflow:Assets written to: dataset/archive/assets


In [68]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('vgg16/') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']

loss = history1.history['loss']
val_loss = history1.history['val_loss']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [65]:
from tensorflow.keras.models import load_model

In [67]:
savedModel=load_model('vgg16/')

savedModel.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 14)                17934     
                                                                 
Total params: 2,275,918
Trainable params: 17,934
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
u=savedModel(X_train)